In [3]:
import numpy as np
import pandas as pd
from collections import Counter
import itertools
import random

# 1. Load Dictionary
def load_words(word_length):
    file_name = f"{word_length}_letter_words.txt"
    with open(file_name, "r", encoding="utf-8") as file:
        return [line.strip().lower() for line in file.readlines()]

# 2. Information-Theoretic Scoring
def calculate_entropy(guess, possible_words):
    feedback_counts = Counter()
    
    for word in possible_words:
        feedback = get_feedback(guess, word)
        feedback_counts[feedback] += 1

    total = sum(feedback_counts.values())
    entropy = -sum((count / total) * np.log2(count / total) for count in feedback_counts.values())
    return entropy

# 3. Feedback Calculation
def get_feedback(guess, target):
    feedback = []
    target_counter = Counter(target)
    
    for i, letter in enumerate(guess):
        if letter == target[i]:
            feedback.append(2)  # Correct position
            target_counter[letter] -= 1
        else:
            feedback.append(0)  # Default to absent
    
    for i, letter in enumerate(guess):
        if feedback[i] == 0 and letter in target_counter and target_counter[letter] > 0:
            feedback[i] = 1  # Misplaced
            target_counter[letter] -= 1
    
    return tuple(feedback)

# 4. Filter Words by Feedback
def filter_words(guess, feedback, possible_words):
    return [word for word in possible_words if get_feedback(guess, word) == feedback]

# 5. Find Best Words Starting with Each Letter
def find_best_words(possible_words):
    best_words = {}
    second_best_words = {}
    turkish_alphabet = 'abcçdefgğhıijklmnoöprsştuüvyz'
    for letter in turkish_alphabet:
        words_starting_with_letter = [word for word in possible_words if word.startswith(letter)]
        if words_starting_with_letter:
            scores = [(guess, calculate_entropy(guess, possible_words)) for guess in words_starting_with_letter]
            scores.sort(key=lambda x: x[1], reverse=True)
            best_words[letter] = scores[0][0]
            if len(scores) > 1:
                second_best_words[letter] = next((word for word in words_starting_with_letter if word[0] == scores[0][0][0] and all(c not in scores[0][0] for c in word[1:])), None)
            else:
                second_best_words[letter] = None
    return best_words, second_best_words



In [6]:
# 6. Run the Solver for different word lengths
for word_length in [4, 5, 6]:
    WORD_LIST = load_words(word_length)
    WORD_LIST = [word.strip() for word in WORD_LIST if len(word.strip()) == word_length]
    POSSIBLE_WORDS = random.sample(WORD_LIST, len(WORD_LIST) // 5)
    
    print(f"{word_length} harfli kelimelerin {len(WORD_LIST)} tanesi yüklendi.")
    
    print(f"Lingo Çözücü - {word_length} harfli kelimeler")
    best_words, second_best_words = find_best_words(WORD_LIST)
    for letter in best_words:
        print(f"'{letter}' ile en iyi tahmin: {best_words[letter]}")
        if second_best_words[letter]:
            print(f"'{letter}' ile ikinci tahmin: {second_best_words[letter]}")
        else:
            print(f"'{letter}' harfi ile ikinci bir tahmin bulunamadı.")

4 harfli kelimelerin 2110 tanesi yüklendi.
Lingo Çözücü - 4 harfli kelimeler
'a' ile en iyi tahmin: aile
'a' ile ikinci tahmin: abuk
'b' ile en iyi tahmin: bari
'b' ile ikinci tahmin: bent
'c' ile en iyi tahmin: cari
'c' ile ikinci tahmin: cebe
'ç' ile en iyi tahmin: çare
'ç' ile ikinci tahmin: çıkı
'd' ile en iyi tahmin: dair
'd' ile ikinci tahmin: defo
'e' ile en iyi tahmin: erat
'e' ile ikinci tahmin: edik
'f' ile en iyi tahmin: fare
'f' ile ikinci tahmin: fıçı
'g' ile en iyi tahmin: gali
'g' ile ikinci tahmin: gebe
'h' ile en iyi tahmin: hare
'h' ile ikinci tahmin: hınç
'ı' ile en iyi tahmin: ırak
'ı' harfi ile ikinci bir tahmin bulunamadı.
'i' ile en iyi tahmin: iare
'i' ile ikinci tahmin: idol
'j' ile en iyi tahmin: jale
'j' ile ikinci tahmin: jips
'k' ile en iyi tahmin: kari
'k' ile ikinci tahmin: kebe
'l' ile en iyi tahmin: laik
'l' ile ikinci tahmin: leçe
'm' ile en iyi tahmin: mira
'm' ile ikinci tahmin: meke
'n' ile en iyi tahmin: nale
'n' ile ikinci tahmin: norm
'o' ile en 

In [10]:
# Load 4-letter words
word_length = 4
WORD_LIST = load_words(word_length)
WORD_LIST = [word.strip() for word in WORD_LIST if len(word.strip()) == word_length]

# Choose a random 4-letter word as the target
target_word = random.choice(WORD_LIST)
print(f"Target word: {target_word}")

# Initialize possible words
possible_words = WORD_LIST.copy()

# Simulate the game
attempts = 0
while True:
  attempts += 1
  if attempts > 5:
    print("Elendiniz!")
    break
  best_words, _ = find_best_words(possible_words)
  best_guess = best_words[target_word[0]]
  feedback = get_feedback(best_guess, target_word)
  
  print(f"Attempt {attempts}: Guess '{best_guess}' -> Feedback {feedback}")
  
  if feedback == (2, 2, 2, 2):
    print(f"Found the word '{target_word}' in {attempts} attempts!")
    break
  

  possible_words = filter_words(best_guess, feedback, possible_words)

Target word: ramp
Attempt 1: Guess 'rika' -> Feedback (2, 0, 0, 1)
Attempt 2: Guess 'raht' -> Feedback (2, 2, 0, 0)
Attempt 3: Guess 'ramp' -> Feedback (2, 2, 2, 2)
Found the word 'ramp' in 3 attempts!


In [27]:
import random
from IPython.display import display, HTML

# Load 4-letter words
word_length = 6
WORD_LIST = load_words(word_length)
WORD_LIST = [word.strip() for word in WORD_LIST if len(word.strip()) == word_length]

# Choose a random 4-letter word as the target
target_word = random.choice(WORD_LIST)
print(f"Target word: {target_word}")

# Initialize possible words
possible_words = WORD_LIST.copy()

# Function to display colored feedback
def display_feedback(guess, feedback):
    colored_feedback = ""
    for letter, status in zip(guess, feedback):
        if status == 2:
            color = "green"
        elif status == 1:
            color = "yellow"
        else:
            color = "grey"
        colored_feedback += f'<span style="color: {color};">{letter}</span>'
    display(HTML(colored_feedback))

# Simulate the game
attempts = 0
for _ in range(5):
    best_words, _ = find_best_words(possible_words)
    best_guess = best_words[target_word[0]]
    feedback = get_feedback(best_guess, target_word)

    display_feedback(best_guess, feedback)

    if feedback == tuple([2] * word_length):
        break

    possible_words = filter_words(best_guess, feedback, possible_words)

Target word: dehliz


In [29]:
# Load words for 4, 5, and 6 letter words
word_lengths = [4, 5, 6]
word_lists = {}

for length in word_lengths:
  word_list = load_words(length)
  word_list = [word.strip() for word in word_list if len(word.strip()) == length]
  word_lists[length] = word_list

# Print the number of words loaded for each length
for length, words in word_lists.items():
  print(f"{length} harfli kelimelerin {len(words)} tanesi yüklendi.")

4 harfli kelimelerin 2110 tanesi yüklendi.
5 harfli kelimelerin 5672 tanesi yüklendi.
6 harfli kelimelerin 6291 tanesi yüklendi.


In [31]:
def find_best_words_and_refine(possible_words):
  best_words = {}
  
  scores = [(guess, calculate_entropy(guess, possible_words)) for guess in possible_words]
  scores.sort(key=lambda x: x[1], reverse=True)
  
  best_guess = scores[0][0]
  best_words[best_guess[0]] = best_guess
  
  return best_words, best_guess


In [38]:
# Load words based on user input for word length and first letter
word_length = int(input("Enter the word length: "))
first_letter = input("Enter the first letter: ").lower()

WORD_LIST = load_words(word_length)
WORD_LIST = [word.strip() for word in WORD_LIST if len(word.strip()) == word_length and word.startswith(first_letter)]

# Initialize possible words
possible_words = WORD_LIST.copy()

# Simulate the game with user input for feedback
attempts = 0
while True:
  attempts += 1
  best_words, _ = find_best_words(possible_words)
  best_guess = best_words[first_letter]
  print(f"Attempt {attempts}: Guess '{best_guess}'")
  
  feedback_input = input(f"Enter feedback for '{best_guess}' (e.g., 22210): ")
  feedback = tuple(int(digit) for digit in feedback_input)
  
  if feedback == tuple([2] * word_length):
    print(f"Found the word in {attempts} attempts!")
    break
  
  possible_words = filter_words(best_guess, feedback, possible_words)

Attempt 1: Guess 'atarlı'
Attempt 2: Guess 'anemik'
Attempt 3: Guess 'anonim'
Found the word in 3 attempts!
